# char-rnn (https://github.com/mineshmathew/char_rnn_karpathy_keras)

In [22]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM,TimeDistributed,SimpleRNN
from keras.utils.data_utils import get_file
import numpy as np
from time import sleep
import random
import sys
maxlen = 40
step = 1
hiddenSizes = [512,512]
dropoutProb = 0.2

### Data

In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

507904/600901 [========================>.....] - ETA: 0s

In [18]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# [a,b,c] --> [b,c,d], [b,c,d]--->[c,d,e]....and so on
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen+1, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i+1:i +1+ maxlen])
print('nb sequences:', len(sentences))

X = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)#dataSize x seqLength x vocabSize
y = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1

for i, sentence in enumerate(next_chars):
    for t, char in enumerate(sentence):
        y[i, t, char_indices[char]] = 1

total chars: 59
nb sequences: 600862


### Model

In [23]:
model = Sequential()
model.add(LSTM(hiddenSizes[0], input_dim=len(chars),return_sequences=True))
for i in xrange(1,len(hiddenSizes)):
    model.add(LSTM(hiddenSizes[1], return_sequences=True))
model.add(Dropout(dropoutProb))
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_11 (LSTM)                   (None, None, 512)     1171456     lstm_input_6[0][0]               
____________________________________________________________________________________________________
lstm_12 (LSTM)                   (None, None, 512)     2099200     lstm_11[0][0]                    
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, None, 512)     0           lstm_12[0][0]                    
____________________________________________________________________________________________________
timedistributed_3 (TimeDistribute(None, None, 59)      30267       dropout_6[0][0]                  
___________________________________________________________________________________________

### Train

In [28]:
history=model.fit(X, y, batch_size=128, nb_epoch=3,verbose=1)

print ('loss is',history.history['loss'][0])

Epoch 1/3
600862/600862 [==============================] - 17814s - loss: 1.2453 
Epoch 2/3
600862/600862 [==============================] - 17812s - loss: 0.7101 
Epoch 3/3
600862/600862 [==============================] - 17841s - loss: 0.5797 
loss is 1.24529485981


### Test

In [37]:
seed_string="brutus:"
print ("seed string -->", seed_string)
print ('The generated text is')
sys.stdout.write(seed_string)
for i in range(100):
    x=np.zeros((1, len(seed_string), len(chars)))
    for t, char in enumerate(seed_string):
        x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
#     next_index=np.argmax(preds[len(seed_string)-1])
    next_index = np.random.choice(range(len(chars)),p=preds[len(seed_string)-1])
    next_char = indices_char[next_index]
    seed_string = seed_string + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()

seed string --> brutus:
The generated text is
brutus:--what? 'e centrilly the belief
existed, perhaps a border, for instance, and makes it not agree-basi

# HRNN (https://github.com/fchollet/keras/blob/master/examples/mnist_hierarchical_rnn.py)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM
batch_size = 32
nb_epochs = 5
row_hidden = 128
col_hidden = 128

## Data

In [ ]:
X_train_ = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test_ = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
row, col, pixel = X_train_.shape[1:]

## Model

In [ ]:
x = Input(shape=(row, col, pixel))
encoded_rows = TimeDistributed(LSTM(output_dim=row_hidden))(x)
encoded_columns = LSTM(col_hidden)(encoded_rows)
prediction = Dense(10, activation='softmax')(encoded_columns)
model = Model(input=x, output=prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train_, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,verbose=1,validation_data=(X_test_, Y_test))

scores = model.evaluate(X_test_, Y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])